In [ ]:
import requests
import pandas as pd 
import time

def get_sp500():
	sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
	sp500_tickers = sp500_tickers[0]

	tickers = sp500_tickers['Symbol'].values.tolist()
	return tickers

companies = get_sp500()
companies.insert(0,'^GSPC')

In [ ]:

print(companies)
api_key = 'your api key'
price = {}
metrics = {}
count = 0
for company in companies:
	count = count + 1
	time.sleep(3)
	print('count ' ,count)
	try:
		prices_retrieval = requests.get(f'https://financialmodelingprep.com/api/v3/historical-price-full/{company}?timeseries=500&apikey={api_key}').json()
		prices_retrieval = prices_retrieval['historical']
		price[company] = {}
		metrics[company] = {}
		for item in prices_retrieval:
			price_date = item['date']
			price[company][price_date] = item['adjClose']
			
		price_DF = pd.DataFrame.from_dict(price)
		price_DF['200_MA'] = price_DF[company].rolling(window=200).mean()
		price_DF['150_MA'] = price_DF[company].rolling(window=150).mean()
		price_DF['50_MA'] = price_DF[company].rolling(window=50).mean()
		price_DF['RS'] = (price_DF[company][–1]/price_DF['^GSPC'][–1] )/ (price_DF[company][–252]/price_DF['^GSPC'][–252]) *100
	
        metrics[company]['200 MA'] = price_DF['200_MA'][–1]
		metrics[company]['150 MA'] = price_DF['150_MA'][–1]
		metrics[company]['50 MA'] = price_DF['50_MA'][–1]
		metrics[company]['200 MA_1mago'] = price_DF['200_MA'][–30]
		metrics[company]['150 MA_1mago'] = price_DF['150_MA'][–30]
		metrics[company]['200 MA_2mago'] = price_DF['200_MA'][–60]
		metrics[company]['150 MA_2mago'] = price_DF['150_MA'][–60]
		metrics[company]['52W_Low'] = price_DF[company][–252:].min()
		metrics[company]['52W_High'] = price_DF[company][–252:].max()
		metrics[company]['price'] = price_DF[company][–1]
		metrics[company]['Relative Strength'] = price_DF['RS'][–1]
		#Current Price is at least 30% above 52 week low (1.3*low_of_52week)
		metrics[company]['Above_30%_low'] = metrics[company]['52W_Low'] *1.3
		# Condition 7: Current Price is within 25% of 52 week high   (.75*high_of_52week)
		metrics[company]['Within_25%_high'] = metrics[company]['52W_High'] * 0.75
	except:
		pass
		


metrics_DF = pd.DataFrame.from_dict(metrics)
metrics_DF = metrics_DF.T 
#to determine the rank percentil and see which are the 80% top performers
metrics_DF['pct_rank'] = metrics_DF['Relative Strength'].rank(pct=True)
metrics_DF = metrics_DF.T
metrics_DF.to_csv('all_stocks_SP500.csv')
print(metrics_DF)

In [ ]:
stocks = pd.read_csv('all_stocks_SP500.csv')
stocks = stocks.T
stocks.columns = stocks.iloc[0]
stocks = stocks[1:]
stocks['condition1'] = (stocks['price'] > stocks['200 MA']) & (stocks['price'] > stocks['150 MA'])

stocks['condition2'] = stocks['150 MA'] > stocks['200 MA']
#3 The 200-day moving average line is trending up for 1 month 
stocks['condition3'] = stocks['200 MA'] > stocks['200 MA_1mago']
stocks['condition4'] = (stocks['50 MA'] > stocks['200 MA']) & (stocks['50 MA'] > stocks['150 MA'])
stocks['condition5'] = stocks['price'] > stocks['50 MA']
#6 The current stock price is at least 30 percent above its 52-week low
stocks['condition6'] = stocks['price'] > stocks['Above_30%_low']
#7 The current stock price is within at least 25 percent of its 52-week high.
stocks['condition7'] = stocks['price'] > stocks['Within_25%_high']
#8 The relative strength ranking is above 80
stocks['condition8'] = stocks['pct_rank'] > 0.8

selection = stocks[(stocks['condition1'] == True) & (stocks['condition2'] == True) & (stocks['condition3'] == True) & (stocks['condition4'] == True)
		& (stocks['condition5'] == True) & (stocks['condition6'] == True) & (stocks['condition7'] == True) & (stocks['condition8'] == True)]

print(selection)